In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Read in dataframe

In [ ]:
df = pd.read_csv('fbdh1.csv')

In [ ]:
# df.head()

In [ ]:
# Use a label fix function to assign classifications or ground truth to Flow thresholds

In [ ]:
def label_fix(label):
    if label < -10:
        return 0
    elif label > 10:
        return 1
    else:
        return 2

df['Class'] = df['Flow'].apply(label_fix)

In [ ]:
df.head()

In [ ]:
# Create the Feature matrix

In [ ]:
X = df.drop('Flow', axis = 1)
X.drop('Class', axis = 1, inplace = True)

In [ ]:
X.head()

In [ ]:
# Scale the features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaler.fit(X)
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features, columns = X.columns[:])

In [ ]:
# Create the classification matrix

In [ ]:
y = df['Class']

In [ ]:
# Perform train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Name columns in tensorflow

In [ ]:
D1 = tf.feature_column.numeric_column('D1')
D2 = tf.feature_column.numeric_column('D2')
P1 = tf.feature_column.numeric_column('P1')
P2 = tf.feature_column.numeric_column('P2')
dD1 = tf.feature_column.numeric_column('dD1')
dD2 = tf.feature_column.numeric_column('dD2')
dP1 = tf.feature_column.numeric_column('dP1')
dP2 = tf.feature_column.numeric_column('dP2')

In [ ]:
feat_cols = [D1, D2, P1, P2, dD1, dD2, dP1, dP2]

In [ ]:
# Create input function for training

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, 
                                                 batch_size = 50, num_epochs = None, shuffle = True)

In [ ]:
# Create Linear Estimator

In [ ]:
Linear = tf.estimator.LinearClassifier(n_classes = 3, feature_columns = feat_cols)

In [ ]:
# Create DNN Estimator

In [ ]:
classifier = tf.estimator.DNNClassifier(hidden_units = [10, 20, 20, 10], 
                                        n_classes = 3, feature_columns = feat_cols, 
                                        model_dir = 
                                        '/Users/mikefurr/Documents/GitHub/BreathDetectionSystem/TensorFlow')

In [ ]:
# Train Linear Estimator

In [ ]:
Linear.train(input_fn = input_func, steps = 5000)

In [ ]:
# Train DNN Estimator

In [ ]:
classifier.train(input_fn = input_func, steps = 5000)

In [ ]:
# Creation Predictions Function

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)

In [ ]:
# Generate note of Linear and DNN Predictions

In [ ]:
note_pred_linear = list(Linear.predict(input_fn = pred_fn))

In [ ]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

In [ ]:
# Gather Predictions in Matrices

In [ ]:
final_linear_preds = []
for pred in note_pred_linear:
    final_linear_preds.append(pred['class_ids'][0])

In [ ]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
# Classification Report Linear Classifier

In [ ]:
print(classification_report(y_test,final_linear_preds))

In [ ]:
# Classification Report DNN Classifier

In [ ]:
print(classification_report(y_test, final_preds))

In [ ]:
# Convert Tensorflow Model to Tensorflow Lite


def serving_input_receiver_fn():
    """
    Serving input_fn that builds features from placeholders

    Returns
    -------
    tf.estimator.export.ServingInputReceiver
    """

    # Create a placeholder for features to load into
    serialized_tf = tf.placeholder(dtype = tf.float32, name = 'input_tensor')

    # Receiving Dictionary of Placeholders for the Features
    receiver_tensors = {'features': serialized_tf}
    
    # Pulls in both the feature placeholders and the features themselves
    features = tf.parse_example(serialized_tf, feature_spec)
    
    # Returns the input receiver object using parse of features and placeholders
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)
    

In [ ]:
feature_columns = [D1, D2, P1, P2, dD1, dD2, dP1, dP2]

In [ ]:
feature_columns

In [ ]:
feature_spec = tf.estimator.classifier_parse_example_spec(
    feature_columns, label_key = 'Class')

In [ ]:
feature_spec

In [ ]:
tf.estimator.export.build_parsing_serving_input_receiver_fn(
    feature_spec,
    default_batch_size=None)

In [ ]:
Linear.export_saved_model('linear_4', 
                              tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec,
                                                                                           default_batch_size=None))

In [ ]:
classifier.export_saved_model('saved_model', 
                              tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec,
                                                                                          default_batch_size=None))

In [103]:
converter = tf.lite.TFLiteConverter.from_saved_model('/Users/mikefurr/Documents/GitHub/BreathDetectionSystem/TensorFlow/saved_model/1567641368/')

In [ ]:
# Instantiate a Keras inception v3 model.
model = tf.keras.applications.inception_v3.InceptionV3(weights = None)

In [ ]:
# Compile model with the optimizer, loss, and metrics you'd like to train with.
model.compile(optimizer = tf.keras.optimizers.SGD(lr = 0.0001, momentum=0.9),
                          loss = 'categorical_crossentropy',
                          metric = 'accuracy')

In [ ]:
# Treat the derived Estimator as you would with any other Estimator.
# First, recover the input name(s) of Keras model, so we can use them as the
# feature column name(s) of the Estimator input function:
model.input_names  # print out: ['input_1']
D1_feat = X['D1'].values
D2_feat = X['D2'].values
P1_feat = X['P1'].values
P2_feat = X['P2'].values
dD1_feat = X['dD1'].values
dD2_feat = X['dD2'].values
dP1_feat = X['dP1'].values
dP2_feat = X['dP2'].values
labels = y.values

In [ ]:
D1_feat

In [ ]:
# for input(s) in the format of Pandas DataFrame:
train_input_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)
    
# To train, we call Estimator's train function:
# Estimator.train(input_fn=train_input_fn, steps=2000)

In [73]:
# Attempt at creating keras model to convert to tensorflow estimator

In [74]:
df.head()

,Flow,D1,D2,P1,P2,dD1,dD2,dP1,dP2,Class
0,-16.69,99.49,-145.71,48.04,-15.90,81.17,12.26,36.71,2.21,0
1,-16.23,172.78,-118.66,79.84,-11.94,80.09,18.49,35.48,2.91,0
2,-15.54,242.66,-81.28,109.77,-7.75,75.74,24.65,33.03,3.26,0
3,-15.17,316.42,-55.50,140.84,-5.30,74.07,28.43,31.89,3.37,0
4,-14.51,392.40,-31.67,171.92,-1.24,73.96,29.70,31.32,3.68,0


In [75]:
b = df.values[:, :9]

In [77]:
def get_features_labels(df):
    
    # Select all Columns but the Last
    features = df.values[:, :9]
    
    # Select Class Column
    labels = df['Class'].values
    
    return features, labels

In [87]:
train_features = X_train.values
train_labels = y_train.values

test_features = X_test.values
test_labels = y_test.values

In [88]:
train_labels_1hot = tf.keras.utils.to_categorical(train_labels)
test_labels_1hot = tf.keras.utils.to_categorical(test_labels)

In [98]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation = tf.nn.relu, input_shape = (8,)))
model.add(tf.keras.layers.Dense(20, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))

In [100]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'rmsprop', 
             metrics = ['accuracy'])

In [101]:
model.summary

<bound method Network.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x1a2c80a3c8>>